In [2]:
import toml
import pandas as pd
from pathlib import Path

# Data Validation

## The validation configuration

*Parameters:*

In [3]:
validation_path = Path.cwd().parent / "data" / "metadata_test" / "_validation_schema_v2.toml"

*Code:*

In [4]:
validation = toml.load(validation_path)
validation

{'metadata_keys': ['type', 'format', 'values'],
 'hosts': {'host_id': {'type': 'string', 'format': 'AA0_00000'},
  'host_groupNumber': {'type': 'integer'},
  'host_sex': {'type': 'string', 'format': 'A', 'values': ['M', 'F']},
  'host_age': {'type': 'integer'},
  'host_death': {'type': 'integer'},
  'host_species': {'type': 'string'},
  'host_breed': {'type': 'string'}},
 'events': {'host_id': {'type': 'string', 'format': 'AA0_00000'},
  'event_day': {'type': 'integer'},
  'event_time': {'type': 'string', 'format': 'HH:MM'},
  'event_type': {'type': 'string',
   'values': ['measurement', 'inoculation', 'treatment']},
  'measurement_type': {'type': 'string'},
  'measurement_quantity': {'type': 'float'},
  'measurement_unit': {'type': 'string'},
  'inoculation_type': {'type': 'string'},
  'inoculation_pathogen': {'type': 'string'},
  'inoculation_dose': {'type': 'float'},
  'inoculation_unit': {'type': 'string'},
  'treatment_type': {'type': 'string'},
  'treatment_dose': {'type': 'float

## Verify data structure in project folder

The project folder should have the follwing structure:

├── data

│ ├── environment.csv

│ ├── environment_events.csv

│ ├── hosts.csv

│ └── host_events.csv

*Parameters*

In [5]:
proj_data_path = Path.cwd().parent / "data" / "synthetic_data"

*Code*

In [77]:
files = [f.name for f in proj_data_path.glob('**/*.csv') if f.is_file()]
files = [f for f in files if f in ['environment_events.csv', 'environment.csv', 'host_events.csv', 'hosts.csv']]
assert "environment.csv" in files
assert "environment_events.csv" in files
assert "hosts.csv" in files
assert "host_events.csv" in files
print(f"Found files: {files}")

Found files: ['environment_events.csv', 'environment.csv', 'host_events.csv', 'hosts.csv']


If all files are found, read them in as pandas dataframes:

In [85]:
data = {}
for f in files:
    try:
        data[f] = pd.read_csv(proj_data_path.joinpath(f))
    except pd.errors.EmptyDataError:
        print(f"File is empty: {f}")
assert list(data.keys()) == files # NOTE: environment_events
print(data.keys())

dict_keys(['environment_events.csv', 'environment.csv', 'host_events.csv', 'hosts.csv'])


## Check column names and each file
### Functions

The function below defines when types in the data frame equal the definition in the validation file:

In [79]:
def type_is_equal(col_type: str, val_type: str) -> bool:
    # string
    if col_type == "object" and val_type == "string":
        return True
    # float
    if col_type.startswith("float") and val_type == "float":
        return True
    # integer
    if col_type.startswith("int") and val_type.startswith("int"):
        return True
    return False

The following function walks over all columns in all csv files and checks their data type. It also notifies about columns that were not found but are defined in the validation file:

In [80]:
def check_column_types(data, validation):
    for data_name in data:
        if data_name == "host_events.csv":
            section = "events"
        else:
            section = data_name.split(".")[0]
        print(f"Checking {data_name} against {section}:")
        for var_name in validation[section]:
            if var_name in data[data_name].columns:
                if not type_is_equal(str(data[data_name][var_name].dtype), validation[section][var_name]["type"]):
                    print(f"{var_name} should be {validation[section][var_name]["type"]}")
                    print(f"\t Found: {str(data[data_name][var_name].dtype)}")
    print("----")

The third function finds all columns which are mentioned in the validation file but cannot be found in the data:

In [83]:
def check_column_exists(data, validation):
    for data_name in data:
        if data_name == "host_events.csv":
            section = "events"
        else:
            section = data_name.split(".")[0]
        for var_name in validation[section]:
            if not var_name in data[data_name].columns:
                print(f"{data_name}; Column not found: {var_name}")
    print("----")

### Check individual files according to section in validation file

In [86]:
check_column_exists(data, validation)

----


In [87]:
check_column_types(data, validation)

Checking environment_events.csv against environment_events:
Checking environment.csv against environment:
Checking host_events.csv against events:
event_day should be integer
	 Found: float64
treatment_unit should be string
	 Found: float64
Checking hosts.csv against hosts:
----


### Check that identifers `host_id` and `environment_id` are defined

### Are identifiers unique?

In [8]:
ids = {}

ids["host_id"] = data["hosts.csv"]["host_id"].unique()
if not len(ids["host_id"]) == len(data["hosts.csv"]["host_id"]):
    print("hosts.csv: Column host_id contains duplicates.")
    assert False

In [15]:
ids["environment_id"] = data["environment.csv"]["environment_id"].unique()
if not len(ids["environment_id"]) == len(data["environment.csv"]["environment_id"]):
    print("environment.csv: Column environment_id contains duplicates.")
    assert False

### Are there None values in the identifier columns?

In [13]:
if sum(pd.isna(ids["host_id"])):
    print(f"hosts.csv: There are {sum(pd.isna(ids['host_id']))} empty cells in host_id.")

hosts.csv: There are 1 empty cells in host_id.


In [17]:
if sum(pd.isna(ids["environment_id"])):
    print(f"environment.csv: There are sum(pd.isna(ids['environment_id'])) empty cells in environment_id.")

### Are all identifiers in the other csv files defined?

In [88]:
for id_type in ids:
    for data_name in data:
        if id_type in data[data_name].columns:
            if not set(data[data_name][id_type]).issubset(ids[id_type]):
                 print(f"File {data_name} contains undefined ids in column {id_type}:")
                 print(set(data[data_name][id_type]).difference(ids[id_type]))

File environment_events.csv contains undefined ids in column host_id:
{'tB5', 'iG2', 'aI5', 'qE1', 'cZ3', 'lJ8', 'yF4', 'xL4', 'iZ9', 'iW3', 'cM9', 'iW1', 'mN5', 'lI2', 'hH9', 'bX7', 'eB2', 'gT8', 'tB2', 'xG1', 'oU1', 'gM8', 'vH4', 'lD2', 'xD2', 'dZ8', 'gD1', 'lP5', 'xK5', 'eA2', 'xW8', 'hC2', 'iM3', 'mH1', 'vK3', 'yZ5', 'hG4', 'pX3', 'qX3', 'fB3', 'sQ9', 'cV6', 'wF0', 'eB9', 'kB0', 'dB4', 'sA5', 'eQ0', 'eK8', 'yL4', 'gZ4', 'yN7', 'bI0', 'hE4', 'dJ8', 'bH2', 'iR5', 'gC7', 'zT4', 'rA3', 'rD6', 'sG1', 'cP1', 'oO2', 'uP0', 'fA7', 'bE4', 'uF5', 'oL5', 'cF6', 'uA2', 'mF9', 'oN2', 'tN6', 'hW5', 'bE5', 'oI4', 'vF8', 'wX3', 'jF8', 'oJ0', 'zV9', 'uQ5', 'aF1', 'gI4', 'dX7', 'mU8', 'bU5', 'qM7', 'uA1', 'pR0', 'vF9', 'xH7', 'uP4', 'eQ5', 'xP9', 'gA8', 'hD8', 'wL3', 'nQ8'}
File host_events.csv contains undefined ids in column host_id:
{'pS7', 'zS6', 'vT4', 'jX6', 'zM4', 'aY6', 'cN7', 'bK0', 'uL0', 'sU0', 'oX3', 'lD9', 'cU5', 'aD7', 'sP4', 'lI7', 'jQ9', 'oH0', 'hH8', 'nL2', 'aH4', 'tX6', 'sK2', 'sI0

## Depedencies between columns

*Parameters:*

Columns that belong together, if one is set, the others also need to be set.

In [121]:
cols = {}
cols["event"] = ["event_day", "event_time", "event_type"]
cols["measurement"] = ["measurement_type", "measurement_quantity", "measurement_unit"]
cols["inoculation"] = ["inoculation_type", "inoculation_pathogen", "inoculation_dose", "inoculation_unit"]
cols["treatment"] = ["treatment_type", "treatment_dose", "treatment_unit"]

*Function:*

In [122]:
def check_column_clusters(cols: dict, data_frame: pd.DataFrame):
    """Check if rows of cluster columns are defined, return row and col names if not."""
    result = []
    for items in cols.values():
        if set(items).issubset(data_frame.columns):
            df = data_frame[items]
            df_na = df[df.isna().any(axis=1)]
            for index, row in df_na.iterrows():
                if not pd.isna(row).all():
                    result.append((index, items))
    return result

### Check for None/NA values in cluster od columns

In [152]:
for data_name in data:
    result = check_column_clusters(cols, data[data_name])
    if len(result) > 0:
        print(f"{data_name}: Invalid entries found:")
        for res in result:
            df = data[data_name][res[1]]
            print(f"\t Line {res[0]}: {df.iloc[[res[0]]]}")

host_events.csv: Invalid entries found:
	 Line 27:     event_day event_time event_type
27        NaN      23:29  treatment
	 Line 32:     event_day event_time   event_type
32       17.0        NaN  measurement
	 Line 0:   treatment_type  treatment_dose  treatment_unit
0  Steven Carter       13.373607             NaN
	 Line 1:   treatment_type  treatment_dose  treatment_unit
1   David Zamora        6.317515             NaN
	 Line 2:   treatment_type  treatment_dose  treatment_unit
2    Maria Gomez         1.48431             NaN
	 Line 3:   treatment_type  treatment_dose  treatment_unit
3    David James        8.969873             NaN
	 Line 4:    treatment_type  treatment_dose  treatment_unit
4  Jennifer Allen        3.441852             NaN
	 Line 5:   treatment_type  treatment_dose  treatment_unit
5  Amanda Bailey        5.075894             NaN
	 Line 6:   treatment_type  treatment_dose  treatment_unit
6   Joshua Rocha        8.286864             NaN
	 Line 7:   treatment_type  trea

## Dependecies between clusters of columns

In [155]:
for data_name in data:
    if set(cols["event"]).issubset(data[data_name].columns):
        measure = set(cols["measurement"]).issubset(data[data_name].columns)
        inoc = set(cols["inoculation"]).issubset(data[data_name].columns)
        treat = set(cols["treatment"]).issubset(data[data_name].columns)
        if not (measure or inoc or treat):
            print(f"{data_name}: Need also information on either of measurement, incoulation or treatment")